In [105]:
import requests
from bs4 import BeautifulSoup
import re
from pymongo import MongoClient
from pymongo.server_api import ServerApi
from bson.objectid import ObjectId
import lxml

In [106]:
URL = 'https://quotes.toscrape.com/'

In [107]:
def get_url_list():
    html_doc = requests.get(URL)
    soup = BeautifulSoup(html_doc.text)
    url_list = []

    content = soup.select('div[class = quote] span a')
    for link in content:
        url = URL + re.search(r'\w+/\S+', link['href']).group()
        url_list.append(url)

    return url_list

get_url_list()

['https://quotes.toscrape.com/author/Albert-Einstein',
 'https://quotes.toscrape.com/author/J-K-Rowling',
 'https://quotes.toscrape.com/author/Albert-Einstein',
 'https://quotes.toscrape.com/author/Jane-Austen',
 'https://quotes.toscrape.com/author/Marilyn-Monroe',
 'https://quotes.toscrape.com/author/Albert-Einstein',
 'https://quotes.toscrape.com/author/Andre-Gide',
 'https://quotes.toscrape.com/author/Thomas-A-Edison',
 'https://quotes.toscrape.com/author/Eleanor-Roosevelt',
 'https://quotes.toscrape.com/author/Steve-Martin']

In [108]:
authors = []
quotes = []

for link in get_url_list():
    html_doc = requests.get(link)
    soup = BeautifulSoup(html_doc.text, 'lxml')
    author_info = {}

    author_details = soup.select_one('div[class = author-details]')

    author_info['fullname'] = author_details.find('h3', {'class':'author-title'}).text
    author_info['born_date'] = author_details.find('span', attrs = {'class':"author-born-date"}).text
    author_info['born_location'] = author_details.find('span', attrs = {'class':"author-born-location"}).text
    author_info['description'] = author_details.find('div', attrs = {'class':"author-description"}).text
        
    authors.append(author_info)


In [109]:

html_doc = requests.get(URL)
soup = BeautifulSoup(html_doc.text)
quotes_details = soup.select('div[class=quote]')

for quote in quotes_details:
    quotes_info = {}
    quotes_info['tags'] = quote.find('meta', attrs={'class':'keywords'}).get('content').split(',')
    quotes_info['authors'] = quote.find('small', attrs={'class':'author'}).text
    quotes_info['quote'] = quote.find('span', attrs={'class':'text'}).text[1:-2]

    quotes.append(quotes_info)


In [110]:
import json

with open ('authors.json', 'w', encoding='utf-8') as f:
    json.dump(authors, f, ensure_ascii=False, indent=4) 


In [111]:
with open ('quotes.json', 'w', encoding='utf-8') as f:
    json.dump(quotes, f, ensure_ascii=False, indent=4) 
